## Modules

In [1]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
from itertools import product
import numpy as np
import pandas as pd

from time import time

## Handling trading days

### Holiday files

In [2]:
# US stock market holidays
with open(os.path.join('holiday_ny.csv'), 'r') as f:
    holidaylines_ny = f.readlines()
    holidaylist_ny = [datetime.strptime(row.split(',')[0], '%Y-%m-%d') for row in holidaylines_ny[1:]]

# HK stock market holidays
with open(os.path.join('holiday_hk.csv'), 'r') as f:
    holidaylines_hk = f.readlines()
    holidaylist_hk = [datetime.strptime(row.split(',')[0], '%Y-%m-%d') for row in holidaylines_hk[1:]]

### Trading days

In [3]:
def getwkdays(startyr=1997, endyr=2046, form='%Y-%m-%d'):
    """Get all working days (non-weekend) to string in a list."""
    dtlist = []
    date = datetime.strptime(f'{startyr}-01-01', '%Y-%m-%d')
    while (date.year >= startyr) and (date.year <= endyr):
        if date.weekday() <= 4:
            dtlist.append(date.strftime(form))
        date += timedelta(days=1)

    return dtlist

workdtlist0 = getwkdays()  #  Format yyyy-mm-dd
workdtlist1 = getwkdays(form='%Y%m%d')  # Format yyyymmdd
workdtlist2 = getwkdays(form='%y%m%d')  # Format yymmdd

In [4]:
def gettradedays(holidaylist, startdt=datetime(2020, 1, 1), enddt=datetime(2022, 12, 31), form='%Y-%m-%d'):
    """Get all trading day spanning a period, excluding holidays."""
    if form == '%Y-%m-%d':
        workdtlist = workdtlist0
    elif form == '%Y%m%d':
        workdtlist = workdtlist1
    elif form == '%y%m%d':
        workdtlist = workdtlist2
    else:
        workdtlist = getwkdays(form=form)
        
    startstr = startdt.strftime(form)
    endstr = enddt.strftime(form)
    holidaystrlist = [date.strftime(form) for date in holidaylist]
    tdlist = [dtstr for dtstr in workdtlist if (dtstr >= startstr) and (dtstr <= endstr)]
    tdlist = [dtstr for dtstr in tdlist if dtstr not in holidaystrlist]

    return tdlist

In [5]:
tdaylist_hk1 = gettradedays(holidaylist_hk)
print(tdaylist_hk1[-20:])

['2022-12-01', '2022-12-02', '2022-12-05', '2022-12-06', '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-12', '2022-12-13', '2022-12-14', '2022-12-15', '2022-12-16', '2022-12-19', '2022-12-20', '2022-12-21', '2022-12-22', '2022-12-23', '2022-12-28', '2022-12-29', '2022-12-30']


### Get latest trading day

In [6]:
def getlatesttradingday(holidaylist, offset=6, form='%Y-%m-%d'):
    """Obtain the latest trading date."""
    today = datetime.today() - timedelta(days=1, hours=offset)
    earlyday = today - timedelta(days=30)
    tdlist = [dtstr for dtstr in gettradedays(holidaylist, earlyday, today, form)]

    return tdlist[-1]

In [7]:
print(getlatesttradingday(holidaylist_ny))
print(getlatesttradingday(holidaylist_hk))

2022-12-01
2022-12-01


### HK Futures monthly settlement days

In [8]:
def gethkexpiry(monthstr='JAN-21'):
    """Obtain HK monthly expiry day in a month."""
    monthstart = datetime.strptime(monthstr, '%b-%y')
    monthstr = monthstart.strftime('%Y-%m')
    year = monthstart.year
    tdaylist = gettradedays(holidaylist_hk, datetime(year, 1, 1), datetime(year, 12, 31))
    tdaylist = [dtstr for dtstr in tdaylist if dtstr[:7] == monthstr] 
    
    return tdaylist[-2]

starttime = time()

expirydict = {}

for year, month in product(range(2007, 2024), range(1, 13)):
    monthstr = (datetime(year, month, 1).strftime('%b-%y')).upper()
    expirydict[monthstr] = gethkexpiry(monthstr)
    
expirylist = list(expirydict.values())
expirylist.sort()

endtime = time()
print(f'Time elapsed getting expiry date dict: {round(endtime - starttime, 4)}s')

Time elapsed getting expiry date dict: 2.4938s


### Read HK settlement day dictionary

In [9]:
starttime = time()

with open('monthexpiry_hk.csv', 'r') as f:
    hkexpirylines = f.readlines()
    hkexpirydict = {}
    for row in hkexpirylines:
        monthstart = datetime.strptime(row[:7], '%Y-%m')
        
        
        monthstr = monthstart.strftime('%b-%y').upper()
        hkexpirydict[monthstr] = row[:-1]
        
hkexpirylist = list(hkexpirydict.values())
hkexpirylist.sort()
    
endtime = time()
print(f'Time elapsed getting expiry date dict: {round(endtime - starttime, 4)}s')


Time elapsed getting expiry date dict: 0.0131s
